In [ ]:
import os, sys, json, datetime
import numpy as np
import skimage.draw

from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Root directory of the project
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)

# Path to trained weights file 
# download weights using "wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5"
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class FootballConfig(Config):
    """Configuration for training on the football dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "football"

    # Adjust this according to your GPU's memory
    IMAGES_PER_GPU = 1

    # Number of classes (including background).
    NUM_CLASSES = 1 + 1  # Background + football

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

class FootballDataset(utils.Dataset):

    def load_Football(self, dataset_dir, subset):
        """Load a subset of the Football dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add football class
        self.add_class("football", 1, "football")

        # Choose from train or validation dataset
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        annotations = json.load(open(os.path.join(dataset_dir, "annot.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']]

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny. Else you could
            # also add the image sizes in the annotation JSONS seperately after VIA labeling.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "football",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)
            
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a football dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "football":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]

        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            # if you end up using any other shape apart from circle or polygon, add the shape format here
            if 'r' in p:
                rr, cc = skimage.draw.circle( p['cx'], p['cy'], p['r'])
                mask[cc, rr,i] = 1
            else:
                rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
                mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

In [ ]:
def train(model, dataset):
    """Train the model."""
    # Training dataset.
    dataset_train = FootballDataset()
    dataset_train.load_Football(dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = FootballDataset()
    dataset_val.load_Football(dataset, "val")
    dataset_val.prepare()

    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')
    
config = FootballConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
dataset = os.path.join(ROOT_DIR , "football_data")

model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])

# train the model
train(model, dataset)

#### Inference

In [ ]:
import matplotlib.pyplot as plt
from mrcnn import visualize

ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR,"logs")
weights_path = os.path.join(MODEL_DIR, "mask_rcnn_football_0030.h5")


class InferenceConfig(FootballConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
# config.display()

# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(weights_path, by_name=True)


# Load a random image from a path (provide your own path here)
image = skimage.io.imread(os.path.join(ROOT_DIR, 'img.jpg'))

# prediction
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
class_names = ['BG', 'football']
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'],
                            title="Predictions")